In [1]:
#from cobra.io import load_matlab_model
import cobra
import numpy as np
import pandas as pd
root_dir = '../../pytfa/'
# Load the model
#model = cobra.io.load_matlab_model(root_dir + 'models/small_ecoli.mat')
# Load reaction DB

In [33]:
model = cobra.io.load_json_model(root_dir + 'models/iJO1366.json')

In [47]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,DM_ca2_e,0.004031,0,0.00%
cbl1_e,DM_cbl1_e,0.0001815,62,0.02%
cl_e,DM_cl_e,0.004031,0,0.00%
cobalt2_e,DM_cobalt2_e,1.954E-05,0,0.00%
cu2_e,DM_cu2_e,0.0005486,0,0.00%
fe2_e,DM_fe2_e,0.01264,0,0.00%
glc-D_e,DM_glc_e,8.16,6,94.18%
glycogenn1_c,DM_glycogenn1_c,0.1255,24,5.79%
k_e,DM_k_e,0.1512,0,0.00%
mg2_e,DM_mg2_e,0.006718,0,0.00%


In [36]:
def single_coenzyme_transform(model,reaction_id):
    met = {}
    met['nad'] = np.where([x.id == 'nad_c' for x in model.metabolites])[0][0]
    met['nadp'] = np.where([x.id == 'nadp_c' for x in model.metabolites])[0][0]
    met['nadh'] = np.where([x.id == 'nadh_c' for x in model.metabolites])[0][0]
    met['nadph'] = np.where([x.id == 'nadph_c' for x in model.metabolites])[0][0]

    met_objs = {}
    met_objs['nad'] = [x for x in model.metabolites if x.id == 'nad_c'][0]
    met_objs['nadh'] = [x for x in model.metabolites if x.id == 'nadh_c'][0]
    met_objs['nadp'] = [x for x in model.metabolites if x.id == 'nadp_c'][0]
    met_objs['nadph'] = [x for x in model.metabolites if x.id == 'nadph_c'][0]
    
    rxn = [x for x in model.reactions if x.id == reaction_id][0].copy()
    # make a new dictionary with coenzyme swapped
    v = {x:y for x,y in rxn.metabolites.items() if x.id in [x + '_c' for x in list(met)]}
    
    nad_stoich = 0;
    nadh_stoich  = 0;
    
    if len(v) > 0:
        v2 = {}
        for x,y in v.items():
            if x.id == 'nadph_c':
                nadh_stoich = nadh_stoich + y 
                #v2[met_objs['nadh']] = y
            elif x.id == 'nadp_c':
                nad_stoich = nad_stoich + y 
                #v2[met_objs['nad']] = y
            elif x.id == 'nadh_c':
                nadh_stoich = nadh_stoich + y 
                #v2[met_objs['nadph']] = y
            elif x.id == 'nad_c':
                nad_stoich = nad_stoich + y 
                #v2[met_objs['nadp']] = y

        v2[met_objs['nad']] = nad_stoich
        v2[met_objs['nadh']] = nadh_stoich

        rxn.subtract_metabolites(v)
        rxn.add_metabolites(v2)
        rxn.id = rxn.id + '[condensed]'
        model.remove_reactions([x for x in model.reactions if x.id == reaction_id][0])
        model.add_reaction(rxn)
        
    return model

In [37]:
#model.reactions.get_by_id('Ec_biomass_iJO1366_WT_53p95M[condensed]')

In [38]:
m  = model.copy()
rxn_ids = [x.id for x in model.reactions]
for rxnid in rxn_ids:
    m = single_coenzyme_transform(m,rxnid)
m.remove_reactions([x for x in m.reactions if x.id == 'NADTRHD[condensed]'][0])

/Users/joshuagoldford/opt/miniconda3/envs/coenzymes/lib/python3.8/site-packages/cobra/core/model.py:745: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [39]:
m.objective = 'Ec_biomass_iJO1366_WT_53p95M[condensed]'

In [40]:
m.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,DM_ca2_e,0.00406,0,0.00%
cbl1_e,DM_cbl1_e,0.0001828,62,0.02%
cl_e,DM_cl_e,0.00406,0,0.00%
cobalt2_e,DM_cobalt2_e,1.968E-05,0,0.00%
cu2_e,DM_cu2_e,0.0005526,0,0.00%
fe2_e,DM_fe2_e,0.01273,0,0.00%
glc-D_e,DM_glc_e,8.16,6,94.14%
glycogenn1_c,DM_glycogenn1_c,0.1264,24,5.83%
k_e,DM_k_e,0.1523,0,0.00%
mg2_e,DM_mg2_e,0.006767,0,0.00%


In [43]:
from pytfa.io import load_thermoDB,                    \
                        read_lexicon, annotate_from_lexicon,            \
                        read_compartment_data, apply_compartment_data
import pytfa

# Load reaction DB
thermo_data = load_thermoDB(root_dir + 'data/thermo_data.thermodb')
lexicon = read_lexicon(root_dir + 'models/iJO1366/lexicon.csv')
compartment_data = read_compartment_data(root_dir + 'models/iJO1366/compartment_data.json')

# Initialize the model
tmodel = pytfa.ThermoModel(thermo_data, m)
tmodel.name = 'NAD_only'

# Annotate the model
annotate_from_lexicon(tmodel, lexicon)
apply_compartment_data(tmodel, compartment_data)

## TFA conversion
tmodel.prepare()
tmodel.convert()

2021-01-31 13:29:07,635 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-01-31 13:29:07,644 - thermomodel_None - INFO - # Model preparation starting...
2021-01-31 13:29:11,902 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 13:29:11,905 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:29:11,906 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:29:11,907 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 13:29:11,910 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:29:11,911 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:29:12,203 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 13:29:12,204 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:29:12,205 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:29:12,207 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 13:29:12,210 - thermomodel_None - WARNING - Warning : NULL/L
2021-01

In [44]:
tmodel.print_info()

                    value
key                      
name             NAD_only
description      NAD_only
num constraints     15022
num variables       15528
num metabolites      1807
num reactions        2584
                             value
key                               
num metabolites(thermo)       1550
num reactions(thermo)         1821
pct metabolites(thermo)  85.777532
pct reactions(thermo)    70.472136


In [45]:
tfa_solution = tmodel.optimize()

In [46]:
tfa_solution

,fluxes,reduced_costs
DM_4CRSOL,0.000000e+00,None
DM_5DRIB,3.013912e-16,None
DM_AACALD,-2.507138e-15,None
DM_AMOB,0.000000e+00,None
DM_MTHTHF,-8.130272e-16,None
...,...,...
UAPGR[condensed],-2.750809e-14,None
UDPGD[condensed],0.000000e+00,None
UDPGDC[condensed],0.000000e+00,None
URDGLYCD[condensed],-5.365381e-14,None


In [48]:
# Initialize the model
tmodel_wt = pytfa.ThermoModel(thermo_data, model)
tmodel_wt.name = 'wt'

# Annotate the model
annotate_from_lexicon(tmodel_wt, lexicon)
apply_compartment_data(tmodel_wt, compartment_data)

## TFA conversion
tmodel_wt.prepare()
tmodel_wt.convert()
tfa_solution_wt = tmodel_wt.optimize()

2021-01-31 14:04:56,087 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-01-31 14:04:56,108 - thermomodel_None - INFO - # Model preparation starting...
2021-01-31 14:05:02,513 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 14:05:02,523 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 14:05:02,542 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 14:05:02,546 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 14:05:02,552 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 14:05:02,558 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 14:05:03,335 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 14:05:03,336 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 14:05:03,337 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 14:05:03,340 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 14:05:03,343 - thermomodel_None - WARNING - Warning : NULL/L
2021-01

In [49]:
tfa_solution_wt

,fluxes,reduced_costs
DM_4CRSOL,0.000000e+00,None
DM_5DRIB,0.000000e+00,None
DM_AACALD,-2.156250e-13,None
DM_AMOB,0.000000e+00,None
DM_MTHTHF,0.000000e+00,None
...,...,...
ZN2abcpp,0.000000e+00,None
ZN2t3pp,0.000000e+00,None
ZN2tpp,0.000000e+00,None
ZNabcpp,0.000000e+00,None
